In [170]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from tqdm import tqdm
from rflearn.env import BlackJack
from rflearn.policy import MCIteration

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [147]:
game = BlackJack(debug=False)
value = pd.Series(0, index=pd.MultiIndex.from_tuples(game.S))
policy = {}
for state in game.S:
    if state[1] >= 20:
        policy[state] = [0, 1]
    else:
        policy[state] = [1, 0]
policy = pd.DataFrame(policy, index=game.A).T

mc_model = MCIteration()
mc_model.fit(game, value, policy, 1)

In [ ]:
policy.copy()

In [164]:
mc_model.transform(10000)

100%|██████████| 10000/10000 [00:05<00:00, 1896.04it/s]


In [165]:
np.round(mc_model.value.unstack(), 1)

1    2    3    4    5    6    7    8    9    10
False 12 -0.6 -0.5 -0.5 -0.5 -0.6 -0.5 -0.5 -0.5 -0.6 -0.6
      13 -0.7 -0.6 -0.6 -0.6 -0.6 -0.6 -0.6 -0.6 -0.5 -0.6
      14 -0.7 -0.6 -0.6 -0.6 -0.6 -0.7 -0.7 -0.6 -0.6 -0.6
      15 -0.8 -0.7 -0.7 -0.6 -0.6 -0.6 -0.7 -0.6 -0.6 -0.7
      16 -0.7 -0.7 -0.6 -0.7 -0.7 -0.6 -0.6 -0.6 -0.7 -0.7
      17 -0.8 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7
      18 -0.8 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.8
      19 -0.8 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7 -0.7
      20  0.1  0.6  0.6  0.7  0.7  0.7  0.8  0.8  0.7  0.4
      21  0.6  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9
True  12 -0.5 -0.3 -0.4 -0.1 -0.2 -0.4 -0.0 -0.2 -0.4 -0.3
      13 -0.6 -0.3 -0.2 -0.3 -0.3 -0.1 -0.2 -0.3 -0.2 -0.4
      14 -0.5 -0.2 -0.4 -0.4 -0.1 -0.3 -0.3 -0.4 -0.3 -0.4
      15 -0.5 -0.4 -0.3 -0.3 -0.3 -0.3 -0.3 -0.3 -0.3 -0.3
      16 -0.6 -0.4 -0.4 -0.3 -0.5 -0.4 -0.3 -0.5 -0.3 -0.4
      17 -0.6 -0.4 -0.5 -0.4 -0.5 -0.4 -0.4 -0.3 -0.2 -0.5
      18 -0.6 -0.4 -0.5 -0.4 -0.3 -0.4 -0.4 -0.4 -0.4 -0.5
      19 -0.7 -0.5 -0.4 -0.5 -0.5 -0.4 -0.4 -0.5 -0.4 -0.5
      20  0.0  0.6  0.6  0.6  0.7  0.7  0.8  0.8  0.8  0.5
      21  0.6  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9  0.9

In [77]:
for _ in tqdm(range(10000)):
    game.deal()
    if not game.get_state() in game.S:
        print(100)

100%|██████████| 100000/100000 [00:07<00:00, 14207.05it/s]
